### **TimesFM**
导读：
TimesFM，一款受NLP启发的创新时间序列（TS）预测模型，其zero-shot性能卓越，接近监督预测模型的水平。借助真实与合成数据的预训练，其解码器式attention机制能精准预测不同领域、范围和粒度的时间序列数据，开启预测新纪元。

#### TimesFM 是基础时间序列预测模型，其拥有如下特质：
* 拥有 2 亿个参数
* 在 1,000 亿个真实世界时间点上进行训练
* 允许将额外的协变量作为特征
* 利用因果self-attention和残差块
* 在zero-shot预测方面优于其他 SOTA 模型

#### TimesFM的作者设法利用了公共时间序列数据中的3个额外资源

* 谷歌趋势：作者将Google Trends搜索兴趣随时间的变化重新组合为时间序列；
* 维基页面浏览量：这些数据记录了维基媒体所有页面的每小时浏览量；
* 合成数据：作者使用ARMA过程创建了一个具有混合季节性、频率和趋势的时间序列语料库。

大型的transformer模型具备强大的学习能力，可以捕获普遍存在的时间模式。然而，面对如何设定上下文长度和预测的时间范围这一挑战，我们陷入了沉思。在horizon长度（即预测长度）的研究中，LLM经过精心优化，已经能够精准地预测下一个单词（基于自回归机制）。但当转向时间序列模型时，却遇到了两大难题：

* 首先，有研究表明，在长期预测任务中，直接预测整个horizon相较于采用多步自回归方法，效果更为显著。这一发现似乎指出了一个更为高效的预测路径
* 其次，在zero-shot场景下，事先无法确定horizon的长度，直接预测整个horizon便显得不切实际。一个理想的通用模型应当具备预测任意时间跨度的能力，以应对各种不确定性和变化。

正是在这样的背景下，TimesFM模型凭借其独特的patching技术，为我们找到了一个平衡点。这一技术在另一个备受瞩目的模型PatchTST中得到了成功应用

TimesFM并非简单地预测单个数据点，也非一次性预测整个horizon长度。相反，它巧妙地将上下文和horizon长度划分为多个patch，从而实现了更为灵活和高效的预测。

这里简单对Patching的工作原理进行阐述：想象一下，我们手头有一段长度为L的上下文内容，以及一个大小为p的patch。这时，我们会将这个输入内容均匀切割成N个片段，每个片段的长度都是p，因此N就等于L除以p。这些切割后的片段，我们称之为输入patch。

除了输入patch，我们还有一个特殊的输出patch，它的大小为h（即预测范围或horizon）。有趣的是，通过让输出patch的大小可以大于输入patch的大小，研究者们惊喜地发现，TimesFM模型能够更迅速、更精确地学会预测任意长度的未来趋势。

下图显示了 TimesFM 在训练过程中的架构：
<center><img src='./pic/timesfm01.png' width=500> </center>


* 首个patch会首先经过输入残差块的处理，这一步的目的是提取输入数据的关键特征。
* 接着，处理后的结果被巧妙地融合到位置编码向量中，这样模型就能理解每个patch在输入数据中的位置信息。
* 然后，步骤2的输出会被送入一个堆叠的transformer。在这个环节中，我们运用了因果self-attention机制，确保每个输出标记只能关注到它之前的输入标记，从而保证了时序数据的顺序性。
* 同样地，transformer的输出会进一步经过输出残差块的处理，最终生成一个输出patch，即我们的预测范围。这个输出patch会与实际的数据进行比较，以计算损失，从而优化模型的预测能力。

在后续的解码步骤中，模型将逐次处理更多的输入patch，并输出对应的输出patch。值得注意的是，在实际操作中，给定输入的所有patch都是在一个训练小批次中一次性创建的，这样可以显著提高训练的效率。

综上所述，我们可以得出结论：如下图的Decoder only LLM所示，TimesFM模型是在解码器模式下进行训练的，这一点与GPT模型相似。然而，它的独特之处在于将每个标记视为一个时间点patch，这种patch处理方式不仅大大提高了推理速度，还使得模型不再受限于特定的预测长度或范围，从而具有更强的灵活性和泛化能力。

<center><img src='./pic/timesfm02.png' width=500></center>
<center><img src='./pic/timesfm03.png' width=500></center>

注：除patch外，还提供了一个屏蔽向量--随机屏蔽patch的一部分。这样做是为了防止模型只学习预测长度为输入patch长度倍数的上下文。




参考：https://mp.weixin.qq.com/s?__biz=MzU2NjU3OTc5NA==&mid=2247583592&idx=1&sn=b8ba61c0e5bbb7144ec0c81cb5b49d3f&chksm=fd2e61a4e7a464a5c090e969018aca8595ddde13e9656f434c8ab5d67680c2d60997991ae2e7&scene=27

#### timesFM模型下载：https://aifasthub.com/models/google/timesfm-1.0-200m
环境安装：